<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<b> Комментарий ревьюера</b>
    
Привет! Меня зовут Светлана Носова, я буду проверять твой проект. Если не против, то давай общаться на «ты». 

Сразу хочу обратить внимание на цветовую разметку.
Зеленым помечены отличные решения и идеи, которые можно и нужно использовать в будущем. Желтым отмечено то, что можно или нужно оптимизировать, в зависимости от контекста. 
    
**Ответы на мои комментарии или доработки по моим рекомендациям попрошу выделить каким-нибудь цветом, кроме трех перечисленных.**
Также я попрошу тебя **не менять, не передвигать и не удалять мои комментарии**, чтобы при последующей проверке мне было проще ориентироваться.
    
Кроме того, комментарии у меня обозначены как заголовки. Это сбивает содержание, но при последующей проверке я могу снять с них заголовок, если тебе так будет удобнее. При этом они удобны, поскольку ты сразу можешь перейти к ним.   
        
<hr>
    
**По проекту:** работа выполнена очень хорошо, все оформлено понятно и аккуратно, есть анализ и промежуточные выводы. У меня остались некоторые вопросы, я все написала в комментариях. Буду ждать работу на повторное ревью.
    
<hr>
Комментарии можешь оставлять с помощью такого кода в Markdown ячейке:
    
    
    <div class="alert alert-info">
    <h2> Комментарий студента</h2>

    Тут текст.
    </div>
  
    
Если не хочешь, чтобы твои комментарии отображались в содержании как заголовки, замени **h2** на **b**.    
</div>

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<b> Комментарий ревьюера 2</b>
    
Я оставила новые комментарии с цифрой 2. Посмотри их, пожалуйста. Работу я могу принять, поскольку вопросов больше нет. Поздравляю с первым решенным проектом и желаю успехов в дальнейшем обучении :)
</div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [78]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter
data.set_axis(['num_children', 'experience', 'years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt_status', 'total_income', 'purpose'], axis='columns', inplace=True)
data_ti = data['total_income', 'income_type']
data_ti

KeyError: ('total_income', 'income_type')

In [2]:
data.describe()

,num_children,experience,years,education_id,family_status_id,debt_status,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
num_children        21525 non-null int64
experience          19351 non-null float64
years               21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt_status         21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
1. Непонятные имена столбцов в соответствии с описанием - нужно переименовать (сделано)
2. Столбец дети - есть отрицательные значения, а также максимально 20 детей (уточнить число случаев)
3. Столбец стаж - отрицательные значения, а также имеются пропуски NaN
4. Не уверен, но очень большой средний стаж mean = 63046.497661 (172 года), возможно, это как-то связано с отрицательными значениями
5. Столбец образование - разный регистр
6. В процессе работы проверить соответствия столбцов образования и семьи с номером статуса (поверхностно всё соответствует)
7. Столбец общего дохода - имеются пропуски NaN
8. Типы данных соответствуют типу значений столбцов, хотя трудовой стаж я бы привёл к int с округлением
9. В столбцах experience и total_income есть пропущенные значения (19351 из 21525), см. пп. 3 и 6.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Хорошая вводная часть.
</div>

In [4]:
print(data[data['num_children'] == -1].count()[0])
print(data[data['num_children'] == 20].count()[0])

47
76


Вывод: оказалось, что число людей с -1 ребёнком 20, а с 20 детьми - 76. Возможно, вместо -1 нужно было вносить 1. 

In [5]:
sum_pens = data[(data['num_children'] > 10) & (data['num_children'] <= 19)]
print(sum_pens.count()[0])

0


Вывод: нет, семьи с детьми - это тоже не многодетные семьи (напр., усыновление, что возможно). В промежутке от 10 до 19 их число - 0, а с 20-ю - 76. Значит, их тоже надо менять (возможно, на 2)

In [6]:
print(data[data['years'] == 0].count()[0])

101


Я и ранее заметил нулевой возраст, это вполне могут быть дети, но дети не берут кредиты. Значит, это ошибка внесения (возраст не указан, например). Для проверки ввёл 1 год, таких значений 0.

In [7]:
print(data[data['experience'] >= 14400].count()[0])

3445


Проверим число записей со стажем более 40 лет - их 3445. Однозначно для доклада директору, пусть разбирается. Возможно, это не дни, а минуты и пр. 

In [8]:
data[(data['experience'].isnull() == True) & (data['total_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
num_children        2174 non-null int64
experience          0 non-null float64
years               2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt_status         2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Кстати, пропущенные записи в стаже и в доходе совпадают, это одни и те же строки.

In [9]:
data.duplicated().sum()

54

In [10]:
data[data.duplicated(keep=False)].sort_values(by=['experience', 'total_income'])

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose
120,0,NaN,46,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,высшее образование
520,0,NaN,35,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,сыграть свадьбу
541,0,NaN,57,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,сделка с подержанным автомобилем
554,0,NaN,60,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,покупка недвижимости
680,1,NaN,30,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,дополнительное образование
21032,0,NaN,60,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,заняться образованием
21132,0,NaN,47,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,ремонт жилью
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка коммерческой недвижимости


Это был экспромт при заполнении предобработки данных, оказалось, это как раз те строки! Дубликаты. Ну, теперь-то уж я точно удалю.

## Шаг 2. Предобработка данных

In [11]:
#data = data.drop_duplicates() Не те дубликаты?
data['experience'] = data['experience'].fillna(value='none')
data.drop(data[data['experience'] == 'none'].index, inplace=True)
data.duplicated().sum()

0

Удаляем строки - дубликаты и проверяем, что они удалены.

In [12]:
data['education'] = data['education'].str.lower()

Заменяем разный тип букв на строчные

In [13]:
data['num_children'] = data['num_children'].replace(-1, 1)

In [14]:
data['num_children'] = data['num_children'].replace(20, 2)
data.head(50)

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.42,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.186,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.2,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.78,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.87,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.76,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [15]:
print(data['years'].median())

42.0


In [16]:
print(data.loc[data['years'] > 70].count())

num_children        98
experience          98
years               98
education           98
education_id        98
family_status       98
family_status_id    98
gender              98
income_type         98
debt_status         98
total_income        98
purpose             98
dtype: int64


Это на всякий случай проверим, нет ли столетних.

In [17]:
data.loc[data['years'] == 0, 'years'] = 42
print(data.head(50))

    num_children experience  years            education  education_id  \
0              1   -8437.67     42               высшее             0   
1              1    -4024.8     36              среднее             1   
2              0   -5623.42     33              среднее             1   
3              3   -4124.75     32              среднее             1   
4              0     340266     53              среднее             1   
5              0   -926.186     27               высшее             0   
6              0    -2879.2     43               высшее             0   
7              0    -152.78     50              среднее             1   
8              2   -6929.87     35               высшее             0   
9              0   -2188.76     41              среднее             1   
10             2   -4171.48     36               высшее             0   
11             0   -792.702     40              среднее             1   
13             0   -1846.64     54  неоконченное вы

Заменим нулевой возраст на медианное значение

In [18]:
data['experience'] = data['experience'].abs()
print(data.head(30))

    num_children experience  years            education  education_id  \
0              1    8437.67     42               высшее             0   
1              1     4024.8     36              среднее             1   
2              0    5623.42     33              среднее             1   
3              3    4124.75     32              среднее             1   
4              0     340266     53              среднее             1   
5              0    926.186     27               высшее             0   
6              0     2879.2     43               высшее             0   
7              0     152.78     50              среднее             1   
8              2    6929.87     35               высшее             0   
9              0    2188.76     41              среднее             1   
10             2    4171.48     36               высшее             0   
11             0    792.702     40              среднее             1   
13             0    1846.64     54  неоконченное вы

В стаже преобразуем значения в положительные

**Вывод**

Итак, удалили дубликаты, привели строки с образованием в один тип.

### Замена типа данных

In [19]:
data['experience'] = data['experience'].astype('int')

In [20]:
data['total_income'] = data['total_income'].astype('int')

In [21]:
data.head(50)

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Заменили на int, выглядит более красиво и понятно.

### Обработка дубликатов

In [22]:
data.duplicated().sum()

0

In [23]:
data[data.duplicated(keep=False)].sort_values(by=['experience', 'total_income'])

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose


**Вывод**

Это я уже перенёс в начало, но ещё раз убедимся в их отсутствии.

### Лемматизация

In [24]:
list_lemmas = []
for k in data['purpose']:
    lemma = m.lemmatize(k)
    list_lemmas.extend(lemma)

unique_lemmas = Counter(list_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 30248),
 ('\n', 19351),
 ('недвижимость', 5723),
 ('покупка', 5353),
 ('жилье', 4035),
 ('автомобиль', 3897),
 ('образование', 3597),
 ('с', 2619),
 ('операция', 2334),
 ('свадьба', 2099),
 ('на', 2016),
 ('свой', 2013),
 ('строительство', 1701),
 ('высокий', 1243),
 ('получение', 1179),
 ('коммерческий', 1178),
 ('для', 1158),
 ('жилой', 1126),
 ('сделка', 844),
 ('дополнительный', 810),
 ('заниматься', 797),
 ('сыграть', 693),
 ('проведение', 685),
 ('сдача', 588),
 ('семья', 570),
 ('собственный', 560),
 ('со', 559),
 ('ремонт', 542),
 ('подержать', 437),
 ('подержанный', 435),
 ('приобретение', 419),
 ('профильный', 389)]

In [25]:
final_list = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 5 if k != ' ' if k != '\n'][0:10]
print(final_list)

['недвижимость', 'покупка', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство', 'высокий', 'получение', 'коммерческий']


Крайне разный разброс слов, взятый по длине более 5 (первые 10 из списка по употреблению).  Можно убрать "высокий" и "получение".

In [26]:
final_list.remove('высокий')
final_list.remove('получение')
final_list.remove('покупка')
final_list.remove('операция')
print(final_list)

['недвижимость', 'автомобиль', 'образование', 'свадьба', 'строительство', 'коммерческий']


<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Операция там влезла из словосочетаний вроде **операции с недвижимостью** :)
</div>

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Удалим и тег "операция". Конечно, хорошо бы было объединить его со стороительством - если они разные. Но вдруг их мало или, наоборот, есть и операции, тогда сравнение будет некорректно. Всё-таки 2334 упоминания. Посмотрим на них, в основном они совпадают с "недвижимостью", поэтому я их опущу.
</div>

In [27]:
data[data['purpose'].str.contains(r'\bоперац')]

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
24,1,338551,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
30,1,335581,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
34,0,4488,35,среднее,1,женат / замужем,0,F,сотрудник,0,104660,операции со своей недвижимостью
59,0,4341,53,среднее,1,гражданский брак,1,F,компаньон,0,261369,операции с недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21483,2,2917,38,начальное,3,гражданский брак,1,F,сотрудник,1,77417,операции с недвижимостью
21490,2,1867,32,среднее,1,женат / замужем,0,F,сотрудник,0,34767,операции с коммерческой недвижимостью
21500,0,578,26,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,77813,операции с коммерческой недвижимостью
21509,0,362161,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью


In [28]:
# КОД РЕВЬЮЕРА
data.purpose

0                       покупка жилья
1             приобретение автомобиля
2                       покупка жилья
3          дополнительное образование
4                     сыграть свадьбу
                     ...             
21520               операции с жильем
21521            сделка с автомобилем
21522                    недвижимость
21523    на покупку своего автомобиля
21524           на покупку автомобиля
Name: purpose, Length: 19351, dtype: object

**Вывод**

Получили конечный список, в дальнейшем его ждёт нумерация.

### Категоризация данных

In [29]:
final_cat = {i:k for k,i in enumerate(final_list)}
print(final_cat)


{'недвижимость': 0, 'автомобиль': 1, 'образование': 2, 'свадьба': 3, 'строительство': 4, 'коммерческий': 5}


In [30]:
data['purpose_id'] = 0
for row in range(len(data)):
    for purpose in final_cat:
        if purpose in data.iloc[row, 11]:
            data.iloc[row, 12] = final_cat[purpose]
        
data.head(50)

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,2
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,2
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0


**Вывод**

Присвоим через энумератор категории и создадим столбец с категориями.

<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid">
<b> Комментарий ревьюера </b>
    
В задаче нас просят обнаружить словари. Словарем можно назвать пару столбцов education и education_id, поскольку они взаимозаменяемы и из них можно слепить словарь (структуру данных dict). Есть еще что-нибудь подобное в таблице?
</div>

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Есть ещё family_status и family_status_id. Если идёт речь о категоризациях, можно было бы обработать (по диапазонам) столбы стажа, возраста, дохода и наличия детей. Ниже я, например, добавлю категории по доходу.
</div>

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
Да. Из них можно создать словарь, как показано ниже. Но мне больше нравится текстовое прдставление информации, поэтому я бы просто один из этих столбцов удалила.
</div>

In [31]:
# КОД РЕВЬЮЕРА
data_family_status = data[['family_status_id', 'family_status']]
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)
data_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
print(data.groupby('num_children')['debt_status'].mean().to_frame().sort_values(by='debt_status'))

              debt_status
num_children             
5                0.000000
3                0.074830
0                0.074902
4                0.088235
1                0.093230
2                0.096455


Зависимость есть, она представлена на таблице, это можно учитывать при принятии решения. Не имеющие детей имеют лучший рейтинг, а что касается 5, вероятно, закралась ошибка.

In [33]:
print(data[data['num_children'] == 5].count()[0])

8


In [34]:
print(data[data['num_children'] == 4].count()[0])

34


Разница существенная, поэтому опустим данные с 5 детьми.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Да, клиентов с 5 детьми просто мало. Кстати, можно еще вот так вывести:
</div>

In [35]:
# КОД РЕВЬЮЕРА
data.groupby('num_children').agg({'debt_status':['count', 'sum', lambda x: x.mean()*100]})

debt_status                
                   count  sum <lambda_0>
num_children                            
0                  12710  952   7.490165
1                   4387  409   9.323000
2                   1918  185   9.645464
3                    294   22   7.482993
4                     34    3   8.823529
5                      8    0   0.000000

In [36]:
# КОД РЕВЬЮЕРА
data.groupby('num_children').agg({'debt_status':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt_status                
                   count  sum <lambda_0>
num_children                            
0                  12710  952      7.49%
1                   4387  409      9.32%
2                   1918  185      9.65%
3                    294   22      7.48%
4                     34    3      8.82%
5                      8    0       0.0%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
print(data.groupby('family_status')['debt_status'].mean().to_frame().sort_values(by='debt_status'))

                       debt_status
family_status                     
вдовец / вдова            0.064740
в разводе                 0.070175
женат / замужем           0.075922
гражданский брак          0.090763
Не женат / не замужем     0.100594


Можно учитывать коэффициенты таблицы. Также видно, что наиболее рискованные категории - это холостые и живущие в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
print(data.groupby('total_income')['debt_status'].mean().to_frame().sort_values(by='debt_status'))

              debt_status
total_income             
145891                0.0
178101                0.0
178104                0.0
178120                0.0
178129                0.0
...                   ...
218742                1.0
180393                1.0
286746                1.0
286566                1.0
20667                 1.0

[18606 rows x 1 columns]


<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
18600 строк.. Следует создать группы по доходам: низкий, ниже среднего, средний и т.д.. Сделаешь?</div>

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Напишем функцию, сортирующую данные по доходу. По данным РБК, средним классом определён человек с доходом 60 - 120 тыс. Что касается низкого уровня, они должен быть ниже МРОТ (13000 - но тут надо уточнить регион банка). Для годового дохода надо умножить месячный доход на 12. Таким образом, разделим так:
</div>

In [39]:
def total_income_id(row):
    if row['total_income'] <= 13000*12:
        return 'низкий'
    elif 50000 < row['total_income'] <= 60000*12:
        return 'ниже среднего'
    elif 120000 < row['total_income'] < 120000*12:
        return 'средний'
    else:
        return 'богатый'

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
Очень хорошо. Теперь у нас есть возрастные группы. Кстати, можно это сделать в самом начале работы, это поможет заполнить пропуски на основе нескольких характеристик клиента. Делается это так:
    
    
    df['total_income'] = df.groupby(['income_type','age_group', 'education'])['total_income'].apply(lambda x: x.fillna(x.median()))

    
Оно же, но без лямбды:
    
    
    df['total_income'] = df['total_income'].fillna(df.groupby(['income_type','age_group', 'education'])['total_income'].transform('median'))


    
Столбец опыта работы нам не так важен, поэтому его можно было бы и не заполнять. А вот доход — да, можно попытаться заполнить. Возьми этот метод себе на заметку.    
</div>

In [40]:
data['total_income_id'] = data.apply(total_income_id, axis=1)
data.head(20)

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose,purpose_id,total_income_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0,ниже среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0,низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,низкий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,2,ниже среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0,ниже среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0,ниже среднего
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0,ниже среднего
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,2,низкий
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0,низкий


**Предыдущий Вывод**
Разница в доходе между менее имущими и более имущими в контексте заложенностей - примерно в 2 раза (за исключением последнего значения). Я бы сделал вывод, что более имущие отдают кредит более неохотно.

In [41]:
print(data.groupby('total_income')['debt_status'].mean().to_frame().sort_values(by='debt_status'))

              debt_status
total_income             
145891                0.0
178101                0.0
178104                0.0
178120                0.0
178129                0.0
...                   ...
218742                1.0
180393                1.0
286746                1.0
286566                1.0
20667                 1.0

[18606 rows x 1 columns]


In [42]:
debt = data.groupby('total_income_id')['debt_status'].mean().to_frame().sort_values(by='debt_status')
print(debt)

                 debt_status
total_income_id             
средний             0.065217
ниже среднего       0.078475
низкий              0.083380
богатый             0.142857


***Новый вывод*** Но здесь тоже мой вывод подтверждён. Богатые имеют больше задолженностей. Люди с уровнем дохода "средний" и "ниже среднего" лучше отдадут кредит.

- Как разные цели кредита влияют на его возврат в срок?

In [79]:
data.groupby('purpose_id')['debt_status'].mean().to_frame().sort_values(by='debt_status')


KeyError: 'purpose_id'

In [44]:
print(final_cat)

{'недвижимость': 0, 'автомобиль': 1, 'образование': 2, 'свадьба': 3, 'строительство': 4, 'коммерческий': 5}


Здесь получается, что приобретатели недвижимости, строители и молодожёны отдают кредит лучше.

## Шаг 4. Общий вывод

Предоставлять кредит надо с учётом бездетности, состоящим в браке, с целью кредита на строительство, недвижимость или свадьбу, семьям с небольшим доходом.

<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid">
<b> Комментарий ревьюера</b>
    
Хорошо. В идеале надо бы привести проценты невозврата в срок у самых надежных и самых безответственных клиентов для каждой группы. Например: самые надежные клиенты это люди, берущие кредит на операции с недвижимостью (такой-то процент невозврата в срок), а самые безответственные те, кто обращается в банк с целью получить деньги на покупку автомобиля (такой-то процент). И так для каждый группы. Надо ведь некий итог работы подвести.
</div>

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 14 columns):
num_children        19351 non-null int64
experience          19351 non-null int64
years               19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt_status         19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
purpose_id          19351 non-null int64
total_income_id     19351 non-null object
dtypes: int64(8), object(6)
memory usage: 2.8+ MB


In [45]:
debt = data.loc[((data['total_income_id'] == 'средний') | (data['total_income_id'] == 'ниже среднего')) & (data['num_children'] == 0) & (data['family_status_id'] == 0) & ((data['purpose_id'] == 0) | (data['purpose_id'] == 3))] 
debt.sort_values(by='debt_status')

,num_children,experience,years,education,education_id,family_status,family_status_id,gender,income_type,debt_status,total_income,purpose,purpose_id,total_income_id
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0,ниже среднего
14267,0,6009,58,высшее,0,женат / замужем,0,M,госслужащий,0,210443,приобретение автомобиля,0,ниже среднего
14263,0,2808,54,среднее,1,женат / замужем,0,F,компаньон,0,279995,жилье,0,ниже среднего
14256,0,940,46,среднее,1,женат / замужем,0,F,сотрудник,0,267994,на покупку автомобиля,0,ниже среднего
14251,0,5736,36,среднее,1,женат / замужем,0,F,сотрудник,0,165196,покупка жилой недвижимости,0,ниже среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3650,0,3876,27,среднее,1,женат / замужем,0,M,госслужащий,1,189882,покупка коммерческой недвижимости,0,ниже среднего
3608,0,155,25,среднее,1,женат / замужем,0,M,сотрудник,1,170455,приобретение автомобиля,0,ниже среднего
17505,0,2194,56,среднее,1,женат / замужем,0,F,компаньон,1,161805,покупка жилой недвижимости,0,ниже среднего
5882,0,367780,64,среднее,1,женат / замужем,0,F,пенсионер,1,252016,операции с жильем,0,ниже среднего


<div class="alert alert-info">
<h2> Комментарий студента</h2>

В принципе, можно объединить всё сортировкой в таблице с учётом изложенного, на основе возвращаемости кредита. Тогда сотруднику можно озвучить, до какой "планки" можно выдавать кредит, или реализовать это программно дополнительным столбцом "да" или "нет" в зависимости от заданной константы.
</div>

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
Хорошая идея.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.